# Imports

In [3]:
import pandas as pd
import numpy as np

In [4]:
import warnings
warnings.filterwarnings('ignore')

# Cleaning

In [5]:
# load the data
train = pd.read_csv('data/train.csv')
store = pd.read_csv('data/store.csv')

array([0, 1])

In [21]:
# different statistics about the data
# train.info()

# check nulls (difference between NaN and 0)
train.isnull().sum()
(train.isnull() & train.ne(0)).sum()

# see rows with NaN
train[train['Store'].isnull()]

# have look at the duplicates - they look crappy we drop them
train[train.duplicated()].head(6)

# check for question marks which are sometimes placeholder for NaN
train.select_dtypes(exclude=np.number).apply( lambda x: x.str.contains("\?", regex=True) ).any()

# what values are in StateHoliday
train.StateHoliday.unique()



train.describe().T 
train.isnull().sum()/len(train)

Date             0.000000
Store            0.030263
DayOfWeek        0.029818
Sales            0.029833
Customers        0.029934
Open             0.030083
Promo            0.030095
StateHoliday     0.030189
SchoolHoliday    0.030320
dtype: float64

# clean and join tables

In [6]:
train.columns

Index(['Date', 'Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday'],
      dtype='object')

# cleaner function

In [33]:
import datetime

def clean_merge(df, store):
    df_ = df.copy()

    df_= df_.drop_duplicates().dropna()

    # convert string Date to datetime
    df_['Date'] = pd.to_datetime(df_.Date, format='%Y-%m-%d')

    # change normal days to 1 and holidays to 0
    df_['StateHoliday'] = df_.StateHoliday.apply(lambda x: 1 if x in ['0', 0.0] else 0)
    
    # join cleaned train and store data 
    df_ = pd.merge(df_, store, how='outer', on='Store')

    # label encode promo Intervals
    d = {'Mar,Jun,Sept,Dec': 1, 'Feb,May,Aug,Nov':2, 'Jan,Apr,Jul,Oct':3, np.nan: 0}
    df_['PromoInterval'] = df_['PromoInterval'].map(d)

    return df_

In [34]:
df_merge = clean_merge(train, store)


In [35]:
# double-checking the new df
# df_merge.StateHoliday.isnull().sum()
# df_merge.StateHoliday.unique()



# df_merge.SchoolHoliday.unique()
# df_merge.StoreType.unique()
# df_merge.Assortment.unique()
# df_merge.CompetitionOpenSinceMonth.unique()
# # df_merge.describe().T

# df_merge.isnull().sum()/len(df_merge)
df_merge.dtypes


Date                         datetime64[ns]
Store                               float64
DayOfWeek                           float64
Sales                               float64
Customers                           float64
Open                                float64
Promo                               float64
StateHoliday                          int64
SchoolHoliday                       float64
StoreType                            object
Assortment                           object
CompetitionDistance                 float64
CompetitionOpenSinceMonth           float64
CompetitionOpenSinceYear            float64
Promo2                                int64
Promo2SinceWeek                     float64
Promo2SinceYear                     float64
PromoInterval                        object
dtype: object

In [ ]:
# convert types

In [36]:
def types_and_imputer(df):
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], downcast='integer')

    # median imputing and converting to int (nobody cares for half a meter to the Competition)
    median_distance = df.CompetitionDistance.median()
    competiton_distance = df.CompetitionDistance.fillna(median_distance).round()
    df.CompetitionDistance = pd.to_numeric(competiton_distance, downcast='integer')
    return df

df_types = types_and_imputer(df_merge)
df_types.dtypes

Date                         datetime64[ns]
Store                                 int16
DayOfWeek                              int8
Sales                                 int32
Customers                             int16
Open                                   int8
Promo                                  int8
StateHoliday                           int8
SchoolHoliday                          int8
StoreType                            object
Assortment                           object
CompetitionDistance                 float64
CompetitionOpenSinceMonth           float64
CompetitionOpenSinceYear            float64
Promo2                                 int8
Promo2SinceWeek                     float64
Promo2SinceYear                     float64
PromoInterval                        object
dtype: object

In [ ]:
def types_and_imputer(df):

    


    df.CompetitionOpenSinceMonth = df.CompetitionOpenSinceMonth.fillna(0).round().astype('int')
    df.CompetitionOpenSinceYear = df.CompetitionOpenSinceYear.fillna(0).round().astype('int')
    df.Promo2SinceWeek = df.Promo2SinceWeek.fillna(0).round().astype('int')
    df.Promo2SinceYear = df.Promo2SinceYear.fillna(0).round().astype('int')

    return df

In [33]:
df_merge.convert_dtypes()
df_dtypes = df_merge.convert_dtypes()
df_dtypes.dtypes

Date                         datetime64[ns]
Store                                 Int64
DayOfWeek                             Int64
Sales                                 Int64
Customers                             Int64
Open                                  Int64
Promo                                 Int64
StateHoliday                         string
SchoolHoliday                         Int64
StoreType                            string
Assortment                           string
CompetitionDistance                   Int64
CompetitionOpenSinceMonth             Int64
CompetitionOpenSinceYear              Int64
Promo2                                Int64
Promo2SinceWeek                       Int64
Promo2SinceYear                       Int64
PromoInterval                        string
dtype: object

# encoding, fillna, datatypes
m, f, j, n

In [25]:
for k, v in  d.items():
  print(f'{k}: {v}')

Mar,Jun,Sept,Dec: m
Feb,May,Aug,Nov: f
Jan,Apr,Jul,Oct: j
nan: n


In [26]:
df_map.PromoInterval.isnull().sum()

0

In [27]:
# downcasting numeric columns for better performance while plotting (hopefully)
for col in df_map.columns:
    if pd.api.types.is_numeric_dtype(df_map[col]):
        df_map[col] = pd.to_numeric(df_map[col], downcast='integer')

In [28]:
# finding the smallestes possible datatype, imputing and handling NA en passant

# df_map2.DayOfWeek = df_map.DayOfWeek.astype('int8')

# median imputing and converting to int (nobody cares for half a meter to the Competition)
median_distance = df_map.CompetitionDistance.median()
competiton_distance = df_map.CompetitionDistance.fillna(median_distance).round()
df_map.CompetitionDistance = pd.to_numeric(competiton_distance, downcast='integer')
df_map.dtypes

Date                         datetime64[ns]
Store                                 int16
DayOfWeek                              int8
Sales                                 int32
Customers                             int16
Open                                   int8
Promo                                  int8
StateHoliday                         object
SchoolHoliday                          int8
StoreType                            object
Assortment                           object
CompetitionDistance                   int32
CompetitionOpenSinceMonth           float64
CompetitionOpenSinceYear            float64
Promo2                                 int8
Promo2SinceWeek                     float64
Promo2SinceYear                     float64
PromoInterval                        object
dtype: object

In [40]:
df_map.Promo2.unique
df_map2 = df_map.copy()

df_map2.Open = df_map2.Open.astype(bool)
df_map2.Promo = df_map2.Promo.astype(bool)
df_map2.Promo2 = df_map2.Promo2.astype(bool)
df_map2.SchoolHoliday = df_map2.SchoolHoliday.astype(bool)
df_map2.CompetitionOpenSinceMonth = df_map2.CompetitionOpenSinceMonth.fillna(0).round().astype('int')
df_map2.CompetitionOpenSinceYear = df_map2.CompetitionOpenSinceYear.fillna(0).round().astype('int')
df_map2.Promo2SinceWeek = df_map2.Promo2SinceWeek.fillna(0).round().astype('int')
df_map2.Promo2SinceYear = df_map2.Promo2SinceYear.fillna(0).round().astype('int')

df_map2.dtypes

Date                         datetime64[ns]
Store                                 int16
DayOfWeek                              int8
Sales                                 int32
Customers                             int16
Open                                   bool
Promo                                  bool
StateHoliday                         object
SchoolHoliday                          bool
StoreType                            object
Assortment                           object
CompetitionDistance                   int32
CompetitionOpenSinceMonth             int64
CompetitionOpenSinceYear              int64
Promo2                                 bool
Promo2SinceWeek                       int64
Promo2SinceYear                       int64
PromoInterval                        object
dtype: object

In [41]:
df_map2.describe().T

,count,mean,std,min,25%,50%,75%,max
Store,499419.0,558.377344,321.781838,1.0,280.0,558.0,837.0,1115.0
DayOfWeek,499419.0,3.994309,1.998609,1.0,2.0,4.0,6.0,7.0
Sales,499419.0,5667.252594,3807.125268,0.0,3645.0,5628.0,7712.0,38037.0
Customers,499419.0,628.731622,463.445275,0.0,398.0,604.0,833.0,7388.0
CompetitionDistance,499419.0,5400.258520,7659.720481,20.0,720.0,2330.0,6880.0,75860.0
CompetitionOpenSinceMonth,499419.0,4.930719,4.283284,0.0,0.0,4.0,9.0,12.0
CompetitionOpenSinceYear,499419.0,1370.955530,935.047362,0.0,0.0,2006.0,2011.0,2015.0
Promo2SinceWeek,499419.0,11.954749,15.473651,0.0,0.0,1.0,22.0,50.0
Promo2SinceYear,499419.0,1023.360471,1005.729904,0.0,0.0,2009.0,2012.0,2015.0


In [42]:
#write the cleaned data to disk
df_map2.to_csv('./mdata/df_merged_optimized.csv',index=False)

In [31]:
df_map2[df_map2['Promo2SinceWeek'].isnull()]
# df_map2.isnull().sum()/len(df_map2)
df_merge.isnull().sum()/len(df_merge)

Date                         0.000000
Store                        0.000000
DayOfWeek                    0.000000
Sales                        0.000000
Customers                    0.000000
Open                         0.000000
Promo                        0.000000
StateHoliday                 0.000000
SchoolHoliday                0.000000
StoreType                    0.000000
Assortment                   0.000000
CompetitionDistance          0.002589
CompetitionOpenSinceMonth    0.317483
CompetitionOpenSinceYear     0.317483
Promo2                       0.000000
Promo2SinceWeek              0.491311
Promo2SinceYear              0.491311
PromoInterval                0.491311
dtype: float64

In [32]:
df_merge.StoreType.unique()

array(['d', 'a', 'c', 'b'], dtype=object)